In [2]:
from pathlib import Path
from cpsat_autotune import (
    tune_for_quality_within_timelimit,
    import_model,
    tune_time_to_optimal,
)

/Users/krupke/anaconda3/envs/mo312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = import_model(Path("models/medium_hg.pb"))

In [5]:
# tune_for_quality_within_timelimit(models[0], 3, 10, "minimize", n_samples_per_param=2, max_samples_per_param=5, n_trials=100)
tune_time_to_optimal(
    model,
    timelimit_in_s=10,
    n_samples_per_param=5,
    max_samples_per_param=10,
    n_trials=100,
)

[I 2024-08-14 16:30:38,709] A new study created in memory with name: no-name-b1133616-b312-4844-b360-a500076ff6d9


Computing baseline
Baseline: [-5.396613, -5.274899, -5.370821, -5.399105, -5.207813, -5.33495, -5.239405, -5.296664, -5.145294, -5.260245]


[I 2024-08-14 16:31:56,933] Trial 0 finished with value: -5.0589584 and parameters: {'repair_hint': False, 'use_lb_relax_lns': False, 'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'linearization_level': 1, 'fp_rounding': 3, 'randomize_search': False, 'diversify_lns_params': False, 'add_objective_cut': False, 'use_objective_lb_search': False, 'use_objective_shaving_search': False, 'search_branching': 0, 'cut_level': 1, 'max_all_diff_cut_size': 1, 'symmetry_level': 2, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'presolve_bve_threshold': 1, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:pr

------------------------------------------------------------
Parameters:
	cp_model_probing_level: 0
	max_all_diff_cut_size: 1
	search_branching: 1
	linearization_level: 2
	repair_hint: True
	max_presolve_iterations: 0
	preferred_variable_order: 2
	presolve_bve_threshold: 1
	randomize_search: True
	diversify_lns_params: True
	ignore_subsolvers: ('lb_tree_search', 'no_lp', 'probing', 'quick_restart_no_lp', 'reduced_costs')
	symmetry_level: 1
	presolve_probing_deterministic_time_limit: 1
	use_lb_relax_lns: True
	fp_rounding: 1
Difference to default: 3.162687899999999
This is significant: True
------------------------------------------------------------
------------------------------------------------------------
Parameters:
	max_all_diff_cut_size: 1
	presolve_probing_deterministic_time_limit: 4
	presolve_bve_threshold: 1
	max_presolve_iterations: 2
Difference to default: 0.23362249999999918
This is significant: True
------------------------------------------------------------
------------

({'cp_model_probing_level': 0,
  'max_all_diff_cut_size': 1,
  'search_branching': 1,
  'linearization_level': 2,
  'repair_hint': True,
  'max_presolve_iterations': 0,
  'preferred_variable_order': 2,
  'presolve_bve_threshold': 1,
  'randomize_search': True,
  'diversify_lns_params': True,
  'ignore_subsolvers': ('lb_tree_search',
   'no_lp',
   'probing',
   'quick_restart_no_lp',
   'reduced_costs'),
  'symmetry_level': 1,
  'presolve_probing_deterministic_time_limit': 1,
  'use_lb_relax_lns': True,
  'fp_rounding': 1},
 3.162687899999999,
 True)